Make a file of grid point distances and elevation biases
===

Import required modules
---

In [9]:
from netCDF4 import Dataset
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AWS metadata
---

In [10]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

CESM LE metadata
---

In [11]:
modelMetaFN = "cesmle_geog_sub.nc"
M = Model(modelMetaFN)
M.loadMeta()

1d dataset


Set up dataframe
---

In [12]:
df = pd.DataFrame(columns=['Site #', 'Site name', 'Site elev',\
                           'Grid Dist Closest', \
                           'Grid Elev Closest', 'Grid Elev Closest Bias', \
                           'Grid Elev Interp',  'Grid Elev Interp Bias'], \
                           index=range(nSites))

AWS info and interpolated elevation from old dataframe
---

In [13]:
dfInfoOld = pd.read_pickle('cesmle_site_info.pkl' )
df['Site #'] = dfInfoOld['Site #']
df['Site name'] = dfInfoOld['Site name']
df['Site elev'] = dfInfoOld['Site elev']
df['Grid Elev Interp'] = dfInfoOld['ElevCorr']
df['Grid Elev Interp Bias'] = dfInfoOld['ElevErr']

Find closest points
---

In [14]:
closestDist = []
closestElev = []
closestBias = []
for awsIX in (range(nSites)):
    awsElev = A.getElev( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    closest = M.closestPoints( awsLatLon )
    d = closest[0]
    px = d['ixLon']
    py = d['ixLat']
    pdist = d['distance']
    plat  = M.getLat2d(py,px)
    plon  = M.getLon2d(py,px)
    pelev = M.getElev(py,px)
    perr = pelev - awsElev
    L = (awsIX, awsElev, pdist, plat, plon, pelev, perr)
#     print "%d %d: M: D = %.2f km, lat/lon/elev = %.2f/%.2f/%.2f m, elev_err = %.2f m" % L
    
    closestDist.append( pdist )
    closestElev.append( pelev )
    closestBias.append( perr )

df['Grid Dist Closest'] = closestDist
df['Grid Elev Closest'] = closestElev
df['Grid Elev Closest Bias'] = closestBias

In [15]:
pd.options.display.float_format = '{:,.2f}'.format

df

,Site #,Site name,Site elev,Grid Dist Closest,Grid Elev Closest,Grid Elev Closest Bias,Grid Elev Interp,Grid Elev Interp Bias
0,1,Swiss Camp,"1,176.00",40.18,"1,066.12",-109.88,912.09,-263.91
1,2,Crawford Point1,"2,022.00",41.55,"1,786.00",-236.00,1896.44,-125.56
2,3,NASA-U,"2,334.00",22.47,"2,051.31",-282.69,2130.69,-203.31
3,4,GITS,"1,869.00",37.69,909.00,-960.00,984.18,-884.82
4,5,Humboldt,"1,995.00",22.30,"1,844.07",-150.93,1762.56,-232.44
5,6,Summit,"3,199.00",51.50,"3,072.80",-126.20,3068.80,-130.20
6,7,Tunu-N,"2,052.00",30.36,"2,153.59",101.59,2114.82,62.82
7,8,DYE-2,"2,099.00",4.77,"2,121.24",22.24,2104.91,5.91
8,9,JAR1,932.00,28.60,564.24,-367.76,742.77,-189.23
9,10,Saddle,"2,467.00",53.72,"2,313.24",-153.76,2348.23,-118.77


Create a netCDF file for the new data
-----

In [16]:
newNCFN = 'dist_elev_meta_cesmle.nc'
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass

ncfile = Dataset(newNCFN,mode='w',format='NETCDF4_CLASSIC') 
ncfile.title = "Closest-point and interpolated grid point metadata"

stn_dim = ncfile.createDimension('station', nSites)

distVar = ncfile.createVariable('distanceClosest', np.float32, ('station'))
distVar.units = "km"
distVar.description = "Distance to nearest grid point"
distVar.long_name = "Distance to nearest grid point"
distVar[:] = list( df['Grid Dist Closest'].astype("float") )

elevClosestVar = ncfile.createVariable('elevationClosest', np.float32, ('station'))
elevClosestVar.units = "m"
elevClosestVar.description = "Elevation at nearest grid point"
elevClosestVar.long_name = "Elevation at nearest grid point"
elevClosestVar[:] = list( df['Grid Elev Closest'].astype("float") )

biasClosestVar = ncfile.createVariable('elevationBiasClosest', np.float32, ('station'))
biasClosestVar.units = "m"
biasClosestVar.description = "Elevation bias at nearest grid point"
biasClosestVar.long_name = "Elevation bias at nearest grid point"
biasClosestVar[:] = list( df['Grid Elev Closest Bias'].astype("float") )

elevInterpVar = ncfile.createVariable('elevationInterp', np.float32, ('station'))
elevInterpVar.units = "m"
elevInterpVar.description = "Interpolated elevation"
elevInterpVar.long_name = "Interpolated elevation"
elevInterpVar[:] = list( df['Grid Elev Interp'].astype("float") )

biasInterpVar = ncfile.createVariable('elevationBiasInterp', np.float32, ('station'))
biasInterpVar.units = "m"
biasInterpVar.description = "Interpolated elevation bias"
biasInterpVar.long_name = "Interpolated elevation bias"
biasInterpVar[:] = list( df['Grid Elev Interp Bias'].astype("float") )

ncfile.close()